In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [2]:
import findspark
findspark.init('../spark-3.2.0-bin-hadoop3.2')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('ML-kmeans').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 03:39:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.ml.clustering import KMeans

In [5]:
data = spark.read.format("libsvm").load("../data/sample_kmeans_data.txt")

24/09/09 03:43:16 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [9]:
kmeans = KMeans().setK(2).setSeed(10)

In [11]:
final_data = data.select('features')

In [12]:
model = kmeans.fit(final_data)

In [14]:
from pyspark.ml.evaluation import ClusteringEvaluator


In [17]:
wssse_values =[]
evaluator = ClusteringEvaluator()
output = model.transform(final_data)   
score = evaluator.evaluate(output)   
wssse_values.append(score)  
print("Silhouette Score:",score)

Silhouette Score: 0.9997530305375207


In [21]:
centers = model.clusterCenters()

In [22]:
centers

[array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]

In [23]:
output

DataFrame[features: vector, prediction: int]

In [24]:
output.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         0|
|(3,[0,1,2],[0.1,0...|         0|
|(3,[0,1,2],[0.2,0...|         0|
|(3,[0,1,2],[9.0,9...|         1|
|(3,[0,1,2],[9.1,9...|         1|
|(3,[0,1,2],[9.2,9...|         1|
+--------------------+----------+

